In [156]:
from mlcomp.data.load import load_classification_test, load_classification_train
from mlcomp.data import preprocess
from mlcomp.util import get_categorical_labels
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier, XGBRFClassifier
from catboost import CatBoostClassifier
import pandas as pd

In [157]:
test = load_classification_test()
test = preprocess.drop_ft2(test)
test = preprocess.remove_outliers(test, handling_method=preprocess.HandlingMethod.CAP_AT_MIN_MAX)
#test = StandardScaler().fit_transform(test)

train = load_classification_train()
train = preprocess.drop_ft2(train)
train = preprocess.remove_outliers(train, handling_method=preprocess.HandlingMethod.CAP_AT_MIN_MAX)

X_train = train.drop(columns='label')
y_train = train['label']

#X_train, y_train = SMOTE().fit_resample(X_train, y_train)

#X_train = StandardScaler().fit_transform(X_train)

In [158]:
values={
  'l2_regularization': 0.010781990505819624,
  'learning_rate': 0.21667124489360576,
  'max_bins': 8,
  'max_depth': 19,
  'max_iter': 132,
  'max_leaf_nodes': 33,
  'min_samples_leaf': 3,
}

clf = HistGradientBoostingClassifier()#categorical_features=get_categorical_labels())
#clf = XGBRFClassifier()
#clf = CatBoostClassifier(eval_metric="TotalF1", per_float_feature_quantization=[f'{x}:border_count=1024' for x in (9, 15, 23)])
# clf = GradientBoostingClassifier()
# clf = SVC()
#clf = MLPClassifier()
clf.set_params(**values)

clf.fit(X_train, y_train)
prediction = clf.predict(test)
df = pd.DataFrame(prediction, columns=["label"])
df.to_csv("pred.csv", index_label="Id")

In [159]:
cross_val_score(clf, X_train, y_train, scoring='f1_macro').mean()

0.7910428493279069